In [ ]:
from pathlib import Path

import os
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
import datetime as dt

import osmnx as ox
from shapely import Point
import folium

# Questions to Explore

## Understand Land use relative to transport
- Where do trips start and end?
- For each hour of the day, which streets are destinations, which are origins. Separate weekday/weekend?
## Reduce Congestion
- Where are the congested zones? What time are they congested? Plot routes which pass through congested zones. Can they be diverted?
- Do the different weight classes have similar congestion zones?
## Reduce car use 
- Compare each trip to the shortest distance. How many trips are far less efficient than the shortest path?
- Which trips could have been replaced by public transport
- Which trips could have been replaced by cycling?

In [ ]:
df_trips = pd.read_parquet("../data/processed/all_trips.parquet")

df_trips.StartDate = pd.to_datetime(df_trips.StartDate)
df_trips.EndDate = pd.to_datetime(df_trips.EndDate)

df_trips["StartHour"] = df_trips.StartDate.dt.hour
df_trips["EndHour"] = df_trips.EndDate.dt.hour
df_trips["Date"] = df_trips.StartDate.dt.date

def classify_day(hour):
        return 'weekend' if hour in [1, 7] else 'weekday'

# Apply the function to StartWDay and EndWDay columns to classify the days
df_trips['StartDayType'] = df_trips['StartWDay'].apply(classify_day)
df_trips['EndDayType'] = df_trips['EndWDay'].apply(classify_day)

# Merge the two classifications into one, considering if the trip starts and ends on the same day
df_trips['DayType'] = df_trips.apply(lambda row: 'weekend' if row['StartDayType'] == 'weekend' and row['EndDayType'] == 'weekend' else 'weekday', axis=1)

In [ ]:
df_trips.columns

# Trip starts and ends within the city

In [ ]:
siegburg = ox.geocode_to_gdf("Siegburg")
siegburg

In [ ]:
m = folium.Map(location=(float(siegburg.lat), float(siegburg.lon)), zoom_level=17)
# boundary
s_boundary = folium.GeoJson(data=siegburg.to_json())
s_boundary.add_to(m)

# centroid
s_centroid = folium.GeoJson(data=Point(0.5*(siegburg["bbox_east"] - siegburg["bbox_west"]) + siegburg["bbox_west"], 0.5*(siegburg["bbox_north"] - siegburg["bbox_south"]) + siegburg["bbox_south"]))
s_centroid.add_to(m)

# Coords
s_coords = folium.GeoJson(data=Point(siegburg["lon"], siegburg["lat"]))
#s_coords.add_to(m)
# m

In [ ]:
start_coords = df_trips.loc[(siegburg.loc[0, "bbox_south"] < df_trips["StartLocLat"]) & (df_trips["StartLocLat"] < siegburg.loc[0, "bbox_north"]) &
                            (siegburg.loc[0, "bbox_west"] < df_trips["StartLocLon"]) & (df_trips["StartLocLon"] < siegburg.loc[0, "bbox_east"]), ["StartLocLat", "StartLocLon"]].apply(lambda row: Point(row["StartLocLat"], row["StartLocLon"]), axis=1)

len(start_coords)

In [ ]:
for p in start_coords[:10]:
    this_p = folium.GeoJson(data=p)
    this_p.add_to(m)
m

# Weight Classes per Hour

In [ ]:
def plot_trip_counts_by_weight_class(df):
    trip_counts = df.loc[:, ["VehicleWeightClass", "StartHour", "DayType", "Date", "Mode"]].groupby(
        ["Date", "VehicleWeightClass", "StartHour", "DayType"]).count().reset_index()

    # Get unique values of DayType
    for i in range(1, 4):
        # Filter data for the specific VehicleWeightClass and DayType
        filtered_data = trip_counts[(trip_counts['VehicleWeightClass'] == i)]
        
        # Append trace for the specific VehicleWeightClass and DayType
        fig = px.box(
            filtered_data,
            x='StartHour',
            y='Mode',
            color="DayType",  
            title=f"Trip starts for Vehichle Weight Class {i}"
        )
        fig.show()

plot_trip_counts_by_weight_class(df_trips)

In [ ]:
H3_RESOLUTION = 5
gdf_trips = gpd.read_parquet(f"../data/processed/all_trips_h3_{H3_RESOLUTION}.parquet")

In [ ]:
gdf_trips.columns

In [ ]:
import pydeck as pdk


COLOR_BREWER_MY_COLOR = [  ## https://colorbrewer2.org/#type=diverging&scheme=PuOr&n=10
    [127 , 59  , 8],
    [179 , 88  , 6],
    [224 , 130 , 20],
    [253 , 184 , 99],
    [254 , 224 , 182],
    [216 , 218 , 235],
    [178 , 171 , 210],
    [128 , 115 , 172],
    [84  , 39  , 136],
    [45  , 0   , 75],
]

heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_trips[["StartLocLon", "StartLocLat"]].sample(frac=0.5),
    opacity=0.7,
    intensity=12,
    get_position=["StartLocLon", "StartLocLat"],
    aggregation=pdk.types.String("SUM"),
    color_range=COLOR_BREWER_MY_COLOR,
    # threshold=1,
    # get_weight="count",
    pickable=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50,
                           longitude=8,
                           zoom=5,
                           bearing=0,
                           pitch=0)

# Render
r = pdk.Deck(layers=[heatmap_layer],
             initial_view_state=view_state,
             # map_style=pdk.map_styles.CARTO_ROAD,
             # tooltip={"html": """<b>Lat, Lon:</b> {lat}, {lon} <br /><b>Count:</b> {count}"""},
            )
r #.to_html("h3_centroids_ColumnLayer.html")